In [7]:
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector
import pandas as pd
import numpy as np

df = pd.read_csv('product_data.csv', index_col= [0])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1853 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1849 non-null   object
 1   product_price  1849 non-null   object
 2   weight         1849 non-null   object
 3   category       1849 non-null   object
 4   EAN            1849 non-null   object
 5   date_added     1849 non-null   object
 6   uuid           1849 non-null   object
 7   removed        1849 non-null   object
 8   product_code   1849 non-null   object
dtypes: object(9)
memory usage: 144.8+ KB


In [8]:
#correct data types
df['product_name'] = df['product_name'].astype('string')
df['category'] = df['category'].astype('string')
df['date_added'] = pd.to_datetime(df['date_added'], infer_datetime_format=True, errors='coerce')
df['EAN'] = df['EAN'].astype('string')
df['uuid'] = df['uuid'].astype('string')
df['removed'] = df['removed'].astype('string')
df['product_code'] = df['product_code'].astype('string')


C:\Users\matts\AppData\Local\Temp\ipykernel_19896\3595168056.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date_added'] = pd.to_datetime(df['date_added'], infer_datetime_format=True, errors='coerce')


In [9]:
#sort weights
df['weight'] = df['weight'].astype('string')
df.dropna(inplace=True, how='all')


In [10]:
def split_weight_units(x):
    try:
        if pd.isna(x):  # Handle missing values directly
            return np.nan, np.nan

        if x[-2:] in ('kg', 'ml', 'oz'):
            unit = x[:-2]
            weight = x[-2:]
        elif x[-2:] == ' .':
            unit = x[:-4]
            weight = x[-4:-2]
        elif x[-1:] == 'g':
            unit = x[:-1]
            weight = x[-1:]
        else:  # If no valid unit found, return np.nan for both
            return np.nan, np.nan

        return unit, weight
    except TypeError:
        return np.nan, np.nan

In [11]:
df['weights_tuple'] = df['weight'].apply(split_weight_units)
df['weights'] = df['weights_tuple'].str[0]
df['unit'] = df['weights_tuple'].str[1]
df['unit'].value_counts()

unit
kg    954
g     881
ml      9
7g      1
oz      1
Name: count, dtype: int64

In [12]:
df.loc[df['weights'].str.match(r'\d+ x \d+' , na=False), 'weights'] = (df.loc[df['weights'].str.match(r'\d+ x \d+', na=False), 'weights'].str.extract(r'(\d+) x (\d+)').astype(int).prod(axis=1)) / 100

In [13]:
df['weights'] = df['weights'].astype('float')
df.loc[df['unit'] == 'g', 'weights'] /= 1000
df.loc[df['unit'] == 'ml', 'weights'] /= 1000
df.loc[df['unit'] == 'oz', 'weights'] /= 35.274

In [14]:
df['weight'] = df['weights']
df.drop(columns=['weights', 'unit', 'weights_tuple'], inplace=True)
df.head()

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.60,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,Tiffany's World Day Out At The Park,£12.99,0.48,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,Tiffany's World Pups Picnic Playset,£7.00,0.59,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,£12.99,0.54,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,£30.00,1.91,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1849 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_name   1849 non-null   string        
 1   product_price  1849 non-null   object        
 2   weight         1846 non-null   float64       
 3   category       1849 non-null   string        
 4   EAN            1849 non-null   string        
 5   date_added     1844 non-null   datetime64[ns]
 6   uuid           1849 non-null   string        
 7   removed        1849 non-null   string        
 8   product_code   1849 non-null   string        
dtypes: datetime64[ns](1), float64(1), object(1), string(6)
memory usage: 144.5+ KB


In [16]:
df['length_of_ean'] = df['EAN'].str.len()
mask = df['EAN'].str.len() > 13
df = df[~mask]
df.head()

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code,length_of_ean
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.60,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h,13
1,Tiffany's World Day Out At The Park,£12.99,0.48,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l,12
2,Tiffany's World Pups Picnic Playset,£7.00,0.59,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v,13
3,Tiffany's World Wildlife Park Adventures,£12.99,0.54,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n,13
4,Cosatto Cosy Dolls Pram,£30.00,1.91,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a,13
